# **PERFORMING IMAGE CLASSIFICATION**

In [ ]:
import os
import pathlib
import glob
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras import Model
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.image import *

# Define function to load the images and labels from a list of file paths

In [ ]:
def load_images_and_labels(image_paths):
  images = []
  labels = []

  for image_path in image_paths:
    image = load_img(image_path, target_size = (32, 32),
                     color_mode = "grayscale")
    image = img_to_array(image)

    label = image_path.split(os.path.sep) [-2]
    label = "positive" in label
    label = float(label)

    images.append(image)
    labels.append(label)
  return np.array(images), np.array(labels)

  """
Notice that we are loading the images in grayscale, and we're encoding the labels by
checking whether the word positive is in the file path of the image
  """

# Define a function to build the neural network.
This model's structure is based on LeNet

In [ ]:
def build_network():
  input_layer = Input(shape = (32, 32, 1))
  x = Conv2D(filters = 20,
             kernel_size = (5, 5),
             padding = "same",
             strides = (1, 1)) (input_layer)
  x = ELU()(x)
  x = BatchNormalization()(x)
  x = MaxPooling2D(pool_size = (2, 2),
                   strides = (2, 2)) (x)
  x = Dropout(0.4)(x)

  x = Conv2D(filters = 50,
             kernel_size = (5, 5),
             padding = "same",
             strides = (1, 1))(x)

  x = ELU()(x)
  x = BatchNormalization()(x)
  x = MaxPooling2D(pool_size = (2, 2),
                   strides = (2, 2))(x)

  x = Dropout(0.4)(x)
  x = Flatten()(x)
  x = Dense(units = 500)(x)
  x = ELU()(x)
  x = Dropout(0.4)(x)

  output = Dense(1, activation = "sigmoid")(x)

  model = Model(inputs = input_layer, outputs= output)
  return model

Because this is a binary classification problem, a single sigmoid-activated neuron is enough in the output layer

# LOAD THE IMAGE PATHS INTO THE LIST

In [ ]:
files_pattern = (pathlib.Path.home() / '.keras' /
                'datasets' /
                'SMILEsmileD-master' / 'SMILEs' / '*'
                / '*' /
                '*.jpg')
files_pattern = str(files_pattern)
dataset_paths = [*glob.glob(files_pattern)]

# use the load_images_and_labels() function defined previously to load the dataset into memory

In [ ]:
x, y = load_images_and_labels(dataset_paths)


# Normalize the images and compute the number of positive, and the total examples in the dataset

In [ ]:
# Normalize the images and compute the number of positive, and the total examples in the dataset
x = x.astype("float32") / 255.0

total = len(y)
total_positive = np.sum(y)
total_negative = total - total_positive

# Create train, test, and validation subsets of the data

In [ ]:
# Create train, test, and validation subsets of the data
(x_train, x_test, y_train, y_test) = train_test_split(x, y,
                                                     test_size=0.15,
                                                     stratify = y,
                                                     random_state = 999)

# Second split for validation set
(x_train, x_val, y_train, y_val) = train_test_split(x_train, y_train,
                                                    test_size=0.2,
                                                    stratify = y_train,
                                                    random_state = 999)


ValueError: With n_samples=0, test_size=0.15 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [ ]:
# Build the model
model = build_network()

# Compile the model
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# Train the model
history = model.fit(x_train, y_train,
                    validation_data=(x_val, y_val),
                    epochs=10, batch_size=32)

NameError: name 'x_train' is not defined